In [14]:
from osgeo import gdal, ogr, osr
import json
import os
import tempfile

def create_pdf(geojson_path, tiff_path, output_pdf):
    # Enable GDAL/OGR exceptions
    gdal.UseExceptions()

    # Create a temporary directory for intermediate files
    temp_dir = tempfile.mkdtemp()

    # Prepare PDF creation options to enable layers
    options = ['WRITE_USERUNIT=YES', 'LAYERS=YES']

    # Create a PDF dataset with layer support
    pdf_ds = gdal.GetDriverByName('PDF').Create(output_pdf, 0, 0, 0, gdal.GDT_Unknown, options)
    if pdf_ds is None:
        raise Exception("Creation of PDF file failed")

    # Add the TIFF image as a raster layer in the PDF
    gdal.Translate(output_pdf, tiff_path, format='PDF', options=['LAYERS=YES', 'OGR_DATASOURCE_LAYERS=YES', 'OGR_DISPLAY_FIELD=YES'])

    # Read GeoJSON file
    geojson_ds = ogr.Open(geojson_path)
    if geojson_ds is None:
        raise Exception("Could not open GeoJSON file")

    # Get spatial reference from GeoJSON
    layer = geojson_ds.GetLayer()
    srs = layer.GetSpatialRef()

    # Create vector layers in PDF
    segments_layer = pdf_ds.CreateLayer("Segments", srs, ogr.wkbPolygon)
    lines_layer = pdf_ds.CreateLayer("Lines", srs, ogr.wkbLineString)
    points_layer = pdf_ds.CreateLayer("Points", srs, ogr.wkbPoint)

    # Add fields to vector layers
    for layer in [segments_layer, lines_layer, points_layer]:
        layer.CreateField(ogr.FieldDefn("class", ogr.OFTString))
        layer.CreateField(ogr.FieldDefn("style", ogr.OFTString))

    # Read and transfer features from GeoJSON to appropriate PDF layers
    with open(geojson_path) as f:
        geojson = json.load(f)

    for feature in geojson['features']:
        layer_type = feature['properties']['layerType']
        geom = ogr.CreateGeometryFromJson(json.dumps(feature['geometry']))
        props = feature['properties']

        if layer_type == 'segment':
            out_feature = ogr.Feature(segments_layer.GetLayerDefn())
            out_feature.SetGeometry(geom)
            style = (f'BRUSH(fc:{props["fillColor"]},id:solid,t:{props["fillOpacity"]});'
                     f'PEN(c:{props["strokeColor"]},w:{props["strokeWidth"]})')
            out_feature.SetField("style", style)
            segments_layer.CreateFeature(out_feature)

        elif layer_type == 'line':
            out_feature = ogr.Feature(lines_layer.GetLayerDefn())
            out_feature.SetGeometry(geom)
            style = f'PEN(c:{props["strokeColor"]},w:{props["strokeWidth"]})'
            out_feature.SetField("style", style)
            lines_layer.CreateFeature(out_feature)

        elif layer_type == 'wellspace':
            out_feature = ogr.Feature(points_layer.GetLayerDefn())
            out_feature.SetGeometry(geom)
            style = f'SYMBOL(c:{props["pointColor"]},id:ogr-sym-0,s:{props["pointRadius"]*10})'
            out_feature.SetField("style", style)
            points_layer.CreateFeature(out_feature)

    # Cleanup
    pdf_ds = None
    geojson_ds = None
    os.rmdir(temp_dir)  # Remove temporary directory



def main():
    tiff_path = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\1627_utm\P2_35A_imagesRGB_orthomosaic.tif"
    geojson_path = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\Results\styled.geojson"
    output_pdf = "output_simple_new.pdf"

    try:
        create_pdf(geojson_path, tiff_path, output_pdf)
        print(f"Successfully created {output_pdf}")
    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()


Error: Too many command options 'OGR_DISPLAY_FIELD=YES'


In [18]:
from osgeo import gdal, ogr, osr
import json
import os
import tempfile

def create_pdf(geojson_path, tiff_path, output_pdf):
    gdal.UseExceptions()
    temp_dir = tempfile.mkdtemp()
    
    # Step 1: Create first PDF with just the image
    image_pdf = os.path.join(temp_dir, "image.pdf")
    try:
        gdal.Translate(image_pdf, tiff_path, format='PDF',
                      creationOptions=[
                          'OGC_PDF=YES',
                          'OGR_PDF_LAYER_NAME=Orthomosaic',
                          'COMPRESS=JPEG',
                          'JPEG_QUALITY=100',
                          'DPI=300'
                      ])
        print("Successfully created image PDF")
    except Exception as e:
        print(f"Error creating image PDF: {str(e)}")
        raise

    # Step 2: Create the final PDF that will contain everything
    pdf_driver = ogr.GetDriverByName('PDF')
    if pdf_driver is None:
        raise Exception("PDF driver not available")

    # Create the PDF with OGC support
    pdf_ds = pdf_driver.CreateDataSource(output_pdf, options=['OGC_PDF=YES'])
    if pdf_ds is None:
        raise Exception("Creation of PDF file failed")

    # Read GeoJSON file
    geojson_ds = ogr.Open(geojson_path)
    if geojson_ds is None:
        raise Exception("Could not open GeoJSON file")

    # Get spatial reference
    layer = geojson_ds.GetLayer()
    srs = layer.GetSpatialRef()

    # Create vector layers
    segments_layer = pdf_ds.CreateLayer("Segments", srs, ogr.wkbPolygon, 
                                      options=['OGR_PDF_GROUP=Vector Layers'])
    lines_layer = pdf_ds.CreateLayer("Lines", srs, ogr.wkbLineString, 
                                    options=['OGR_PDF_GROUP=Vector Layers'])
    points_layer = pdf_ds.CreateLayer("Points", srs, ogr.wkbPoint, 
                                     options=['OGR_PDF_GROUP=Vector Layers'])

    # Add fields
    for layer in [segments_layer, lines_layer, points_layer]:
        layer.CreateField(ogr.FieldDefn("class", ogr.OFTString))
        layer.CreateField(ogr.FieldDefn("style", ogr.OFTString))

    # Process GeoJSON features
    with open(geojson_path) as f:
        geojson = json.load(f)

    print("Adding vector features...")
    for feature in geojson['features']:
        layer_type = feature['properties']['layerType']
        geom = ogr.CreateGeometryFromJson(json.dumps(feature['geometry']))

        if layer_type == 'segment':
            out_feature = ogr.Feature(segments_layer.GetLayerDefn())
            out_feature.SetGeometry(geom)
            props = feature['properties']
            style = (f'BRUSH(fc:{props["fillColor"]},id:solid,t:{props["fillOpacity"]});'
                    f'PEN(c:{props["strokeColor"]},w:{props["strokeWidth"]})')
            out_feature.SetField("style", style)
            segments_layer.CreateFeature(out_feature)
        elif layer_type == 'line':
            out_feature = ogr.Feature(lines_layer.GetLayerDefn())
            out_feature.SetGeometry(geom)
            props = feature['properties']
            style = f'PEN(c:{props["strokeColor"]},w:{props["strokeWidth"]})'
            out_feature.SetField("style", style)
            lines_layer.CreateFeature(out_feature)
        elif layer_type == 'wellspace':
            out_feature = ogr.Feature(points_layer.GetLayerDefn())
            out_feature.SetGeometry(geom)
            props = feature['properties']
            style = f'SYMBOL(c:{props["pointColor"]},id:ogr-sym-0,s:{props["pointRadius"]*10})'
            out_feature.SetField("style", style)
            points_layer.CreateFeature(out_feature)

    print("Adding raster layer...")
    # Add the image PDF as a layer
    pdf_ds.SetMetadata({
        'LAYER_OPTIONS': f'EXTRA_IMAGES={image_pdf}',
        'OGR_PDF_RASTER_GROUP': 'Raster Layers'
    }, 'LAYER_METADATA')

    # Cleanup
    pdf_ds = None
    geojson_ds = None
    
    # Remove temporary files
    try:
        os.remove(image_pdf)
        os.rmdir(temp_dir)
        print("Cleanup completed")
    except Exception as e:
        print(f"Warning during cleanup: {str(e)}")

def main():
    tiff_path = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\1627_utm\P2_35A_imagesRGB_orthomosaic.tif"
    geojson_path = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\Results\styled.geojson"
    output_pdf = "output_final.pdf"
    
    try:
        create_pdf(geojson_path, tiff_path, output_pdf)
        print(f"Successfully created {output_pdf}")
    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()

Successfully created image PDF
Adding vector features...
Adding raster layer...
Cleanup completed
Successfully created output_final.pdf


In [27]:
from osgeo import gdal, ogr, osr
import json
import os
import tempfile

def create_pdf(geojson_path, tiff_path, output_pdf):
    # Enable GDAL/OGR exceptions
    gdal.UseExceptions()

    # Create a temporary directory for intermediate files
    temp_dir = tempfile.mkdtemp()
    
    # First create base PDF with the TIFF image
    base_layer_path = os.path.join(temp_dir, "base.pdf")
    gdal.Translate(base_layer_path, tiff_path, format='PDF')

    # Create PDF driver
    pdf_driver = ogr.GetDriverByName('PDF')
    if pdf_driver is None:
        raise Exception("PDF driver not available")

    # Create the PDF dataset
    pdf_ds = pdf_driver.CreateDataSource(output_pdf)
    if pdf_ds is None:
        raise Exception("Creation of PDF file failed")

    # Read GeoJSON file
    geojson_ds = ogr.Open(geojson_path)
    if geojson_ds is None:
        raise Exception("Could not open GeoJSON file")

    # Get spatial reference from GeoJSON
    layer = geojson_ds.GetLayer()
    srs = layer.GetSpatialRef()

    # Create layers in PDF
    segments_layer = pdf_ds.CreateLayer("Segments", srs, ogr.wkbPolygon)
    lines_layer = pdf_ds.CreateLayer("Lines", srs, ogr.wkbLineString)
    points_layer = pdf_ds.CreateLayer("Points", srs, ogr.wkbPoint)

    # Add fields
    for layer in [segments_layer, lines_layer, points_layer]:
        layer.CreateField(ogr.FieldDefn("class", ogr.OFTString))
        layer.CreateField(ogr.FieldDefn("style", ogr.OFTString))

    # Read features from GeoJSON
    with open(geojson_path) as f:
        geojson = json.load(f)

    for feature in geojson['features']:
        layer_type = feature['properties']['layerType']

        # Create feature geometry
        geom = ogr.CreateGeometryFromJson(json.dumps(feature['geometry']))

        # Create feature in appropriate layer
        if layer_type == 'segment':
            out_feature = ogr.Feature(segments_layer.GetLayerDefn())
            out_feature.SetGeometry(geom)
            props = feature['properties']
            style = (f'BRUSH(fc:{props["fillColor"]},id:solid,t:{props["fillOpacity"]});'
                    f'PEN(c:{props["strokeColor"]},w:{props["strokeWidth"]})')
            out_feature.SetField("style", style)
            segments_layer.CreateFeature(out_feature)

        elif layer_type == 'line':
            out_feature = ogr.Feature(lines_layer.GetLayerDefn())
            out_feature.SetGeometry(geom)
            props = feature['properties']
            style = f'PEN(c:{props["strokeColor"]},w:{props["strokeWidth"]})'
            out_feature.SetField("style", style)
            lines_layer.CreateFeature(out_feature)

        elif layer_type == 'wellspace':
            out_feature = ogr.Feature(points_layer.GetLayerDefn())
            out_feature.SetGeometry(geom)
            props = feature['properties']
            style = f'SYMBOL(c:{props["pointColor"]},id:ogr-sym-0,s:{props["pointRadius"]*10})'
            out_feature.SetField("style", style)
            points_layer.CreateFeature(out_feature)

    # Add base TIFF image by merging the base PDF
    tiff_ds = gdal.Open(tiff_path)
    if tiff_ds is not None:
        pdf_ds.SetMetadata({'LAYER_OPTIONS': f'RASTER_IMAGE={base_layer_path}'})

    # Cleanup
    pdf_ds = None
    geojson_ds = None
    tiff_ds = None
    os.remove(base_layer_path)  # Clean up temporary file
    os.rmdir(temp_dir)  # Remove temporary directory

def main():
    tiff_path = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\1627_utm\P2_35A_imagesRGB_orthomosaic.tif"
    geojson_path = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\Results\styled.geojson"
    output_pdf = "output_simple.pdf"

    try:
        create_pdf(geojson_path, tiff_path, output_pdf)
        print(f"Successfully created {output_pdf}")
    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()

Successfully created output_simple.pdf


In [30]:
from osgeo import gdal, ogr, osr
import json
import os
import tempfile

def create_pdf(geojson_path, tiff_path, output_pdf):
    # Enable GDAL/OGR exceptions
    gdal.UseExceptions()

    # Create a temporary directory for intermediate files
    temp_dir = tempfile.mkdtemp()
    
    # Create PDF driver
    pdf_driver = ogr.GetDriverByName('PDF')
    if pdf_driver is None:
        raise Exception("PDF driver not available")

    # Create the PDF dataset with specific creation options to enable layers
    creation_options = ['OGR_WRITE_LAYERS_AS_OPTIONAL=YES']
    pdf_ds = pdf_driver.CreateDataSource(output_pdf, options=creation_options)
    if pdf_ds is None:
        raise Exception("Creation of PDF file failed")

    # Read GeoJSON file
    geojson_ds = ogr.Open(geojson_path)
    if geojson_ds is None:
        raise Exception("Could not open GeoJSON file")

    # Get spatial reference from GeoJSON
    layer = geojson_ds.GetLayer()
    srs = layer.GetSpatialRef()

    # Create vector layers in PDF
    segments_layer = pdf_ds.CreateLayer("Segments", srs, ogr.wkbPolygon, 
                                      options=['OGR_DISPLAY_LAYER_ORDER=4'])
    lines_layer = pdf_ds.CreateLayer("Lines", srs, ogr.wkbLineString,
                                    options=['OGR_DISPLAY_LAYER_ORDER=3'])
    points_layer = pdf_ds.CreateLayer("Points", srs, ogr.wkbPoint,
                                    options=['OGR_DISPLAY_LAYER_ORDER=2'])

    # Add fields
    for layer in [segments_layer, lines_layer, points_layer]:
        layer.CreateField(ogr.FieldDefn("class", ogr.OFTString))
        layer.CreateField(ogr.FieldDefn("style", ogr.OFTString))

    # Read features from GeoJSON
    with open(geojson_path) as f:
        geojson = json.load(f)

    # Process vector features
    for feature in geojson['features']:
        layer_type = feature['properties']['layerType']

        # Create feature geometry
        geom = ogr.CreateGeometryFromJson(json.dumps(feature['geometry']))

        # Create feature in appropriate layer
        if layer_type == 'segment':
            out_feature = ogr.Feature(segments_layer.GetLayerDefn())
            out_feature.SetGeometry(geom)
            props = feature['properties']
            style = (f'BRUSH(fc:{props["fillColor"]},id:solid,t:{props["fillOpacity"]});'
                    f'PEN(c:{props["strokeColor"]},w:{props["strokeWidth"]})')
            out_feature.SetField("style", style)
            segments_layer.CreateFeature(out_feature)

        elif layer_type == 'line':
            out_feature = ogr.Feature(lines_layer.GetLayerDefn())
            out_feature.SetGeometry(geom)
            props = feature['properties']
            style = f'PEN(c:{props["strokeColor"]},w:{props["strokeWidth"]})'
            out_feature.SetField("style", style)
            lines_layer.CreateFeature(out_feature)

        elif layer_type == 'wellspace':
            out_feature = ogr.Feature(points_layer.GetLayerDefn())
            out_feature.SetGeometry(geom)
            props = feature['properties']
            style = f'SYMBOL(c:{props["pointColor"]},id:ogr-sym-0,s:{props["pointRadius"]*10})'
            out_feature.SetField("style", style)
            points_layer.CreateFeature(out_feature)

    # Create a new layer for the TIFF image with specific options
    raster_layer = pdf_ds.CreateLayer("Orthomosaic", srs, ogr.wkbNone,
                                    options=['OGR_DISPLAY_LAYER_ORDER=1'])
    
    # Add TIFF image as a separate layer
    tiff_ds = gdal.Open(tiff_path)
    if tiff_ds is not None:
        # Create temporary PDF with just the TIFF using specific options
        temp_tiff_pdf = os.path.join(temp_dir, "tiff_layer.pdf")
        
        translate_options = gdal.TranslateOptions(
            format='PDF',
            outputSRS=srs,
            creationOptions=[
                'COMPRESS=JPEG',
                'JPEG_QUALITY=90',
                'GEO_ENCODING=ISO32000'
            ]
        )
        
        gdal.Translate(temp_tiff_pdf, tiff_path, options=translate_options)
        
        # Set the TIFF PDF as an optional layer with specific metadata
        pdf_ds.SetMetadata({
            'LAYER_OPTIONS': f'RASTER_IMAGE={temp_tiff_pdf}',
            'OGR_PDF_OPTIONAL_LAYERS': 'YES',
            'OGR_PDF_LAYER_GROUP': '/BaseLayer/Orthomosaic'
        })
        
        # Keep the temporary PDF until the final PDF is written
        # We'll remove it in cleanup

    # Cleanup
    pdf_ds = None
    geojson_ds = None
    tiff_ds = None
    
    # # Remove temporary files after the PDF has been written
    # if os.path.exists(os.path.join(temp_dir, "tiff_layer.pdf")):
    #     os.remove(os.path.join(temp_dir, "tiff_layer.pdf"))
    # os.rmdir(temp_dir)  # Remove temporary directory

def main():
    tiff_path = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\1627_utm\P2_35A_imagesRGB_orthomosaic.tif"
    geojson_path = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\Results\styled.geojson"
    output_pdf = "output_with_layers_NEW.pdf"

    try:
        create_pdf(geojson_path, tiff_path, output_pdf)
        print(f"Successfully created {output_pdf}")
    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()

Successfully created output_with_layers_NEW.pdf


In [34]:
from osgeo import gdal, ogr, osr
import json
import os
import tempfile

def create_pdf(geojson_path, tiff_path, output_pdf):
    # Enable GDAL/OGR exceptions
    gdal.UseExceptions()

    # Create a temporary directory for intermediate files
    temp_dir = tempfile.mkdtemp()
    
    # First create a separate PDF with just the TIFF image
    tiff_pdf_path = os.path.join(temp_dir, "tiff_base.pdf")
    
    # Open TIFF dataset to get its spatial reference
    tiff_ds = gdal.Open(tiff_path)
    if tiff_ds is None:
        raise Exception("Could not open TIFF file")
    
    # Get the spatial reference from the TIFF
    tiff_srs = osr.SpatialReference()
    tiff_srs.ImportFromWkt(tiff_ds.GetProjection())
    
    # Convert TIFF to PDF with specific options
    translate_options = gdal.TranslateOptions(
        format='PDF',
        creationOptions=[
            'DPI=300',
            'COMPRESS=JPEG',
            'JPEG_QUALITY=90',
            'GEOMETRY_ENCODING=ISO32000',
            'GEO_ENCODING=ISO32000',
            'MARGIN=0',
            'EXTRA_IMAGES=NO',
            'WRITE_USERUNIT=YES'
        ]
    )
    
    gdal.Translate(tiff_pdf_path, tiff_path, options=translate_options)
    
    # Create PDF driver for vector layers
    pdf_driver = ogr.GetDriverByName('PDF')
    if pdf_driver is None:
        raise Exception("PDF driver not available")

    # Create the PDF dataset with specific options
    vector_pdf_path = os.path.join(temp_dir, "vector_layers.pdf")
    creation_options = [
        'OGR_WRITE_LAYERS_AS_OPTIONAL=YES',
        'STREAM_COMPRESS=DEFLATE',
        'EXTRA_STREAM_ENCODING=COMPRESS'
    ]
    pdf_ds = pdf_driver.CreateDataSource(vector_pdf_path, options=creation_options)
    if pdf_ds is None:
        raise Exception("Creation of vector PDF file failed")

    # Read GeoJSON file
    geojson_ds = ogr.Open(geojson_path)
    if geojson_ds is None:
        raise Exception("Could not open GeoJSON file")
    
    # Get spatial reference from GeoJSON
    layer = geojson_ds.GetLayer()
    srs = layer.GetSpatialRef()

    # Create vector layers in PDF
    segments_layer = pdf_ds.CreateLayer("Segments", srs, ogr.wkbPolygon, 
                                      options=['OGR_DISPLAY_LAYER_ORDER=4'])
    lines_layer = pdf_ds.CreateLayer("Lines", srs, ogr.wkbLineString,
                                    options=['OGR_DISPLAY_LAYER_ORDER=3'])
    points_layer = pdf_ds.CreateLayer("Points", srs, ogr.wkbPoint,
                                    options=['OGR_DISPLAY_LAYER_ORDER=2'])

    # Add fields
    for layer in [segments_layer, lines_layer, points_layer]:
        layer.CreateField(ogr.FieldDefn("class", ogr.OFTString))
        layer.CreateField(ogr.FieldDefn("style", ogr.OFTString))

    # Read features from GeoJSON
    with open(geojson_path) as f:
        geojson = json.load(f)

    # Process vector features
    for feature in geojson['features']:
        layer_type = feature['properties']['layerType']
        geom = ogr.CreateGeometryFromJson(json.dumps(feature['geometry']))

        if layer_type == 'segment':
            out_feature = ogr.Feature(segments_layer.GetLayerDefn())
            out_feature.SetGeometry(geom)
            props = feature['properties']
            style = (f'BRUSH(fc:{props["fillColor"]},id:solid,t:{props["fillOpacity"]});'
                    f'PEN(c:{props["strokeColor"]},w:{props["strokeWidth"]})')
            out_feature.SetField("style", style)
            segments_layer.CreateFeature(out_feature)

        elif layer_type == 'line':
            out_feature = ogr.Feature(lines_layer.GetLayerDefn())
            out_feature.SetGeometry(geom)
            props = feature['properties']
            style = f'PEN(c:{props["strokeColor"]},w:{props["strokeWidth"]})'
            out_feature.SetField("style", style)
            lines_layer.CreateFeature(out_feature)

        elif layer_type == 'wellspace':
            out_feature = ogr.Feature(points_layer.GetLayerDefn())
            out_feature.SetGeometry(geom)
            props = feature['properties']
            style = f'SYMBOL(c:{props["pointColor"]},id:ogr-sym-0,s:{props["pointRadius"]*10})'
            out_feature.SetField("style", style)
            points_layer.CreateFeature(out_feature)

    # Close the datasets
    pdf_ds = None
    geojson_ds = None
    tiff_ds = None

    # Now merge the TIFF PDF and vector PDF using gdal_merge.py
    merge_command = f'gdal_merge.py -o {output_pdf} {tiff_pdf_path} {vector_pdf_path}'
    os.system(merge_command)
    print(vector_pdf_path)
    print(tiff_pdf_path)

    # # Cleanup temporary files
    # if os.path.exists(tiff_pdf_path):
    #     os.remove(tiff_pdf_path)
    # if os.path.exists(vector_pdf_path):
    #     os.remove(vector_pdf_path)
    # os.rmdir(temp_dir)

def main():
    tiff_path = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\1627_utm\P2_35A_imagesRGB_orthomosaic.tif"
    geojson_path = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\Results\styled.geojson"
    output_pdf = "output.pdf"

    try:
        create_pdf(geojson_path, tiff_path, output_pdf)
        print(f"Successfully created {output_pdf}")
    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()

C:\Users\User\AppData\Local\Temp\tmptultbtls\vector_layers.pdf
C:\Users\User\AppData\Local\Temp\tmptultbtls\tiff_base.pdf
Successfully created output.pdf


In [38]:
from osgeo import gdal
import fitz
import os
import numpy as np

def create_georeferenced_pdf(input_tiff, output_pdf):
    """
    Convert a georeferenced TIFF to a PDF with toggleable layers while preserving georeference
    
    Args:
        input_tiff (str): Path to input georeferenced TIFF file
        output_pdf (str): Path for output PDF file
    """
    # Open the input TIFF
    tiff_ds = gdal.Open(input_tiff)
    if tiff_ds is None:
        raise Exception("Could not open TIFF file")

    # Get geotransform information
    geotransform = tiff_ds.GetGeoTransform()
    projection = tiff_ds.GetProjection()

    # Create temporary GeoTIFF with world file
    temp_tiff = "temp_output.tif"
    gdal_translate_options = gdal.TranslateOptions(
        format="GTiff",
        creationOptions=["COMPRESS=LZW"],
        outputSRS=projection,
        GCPs=tiff_ds.GetGCPs()
    )
    
    gdal.Translate(temp_tiff, tiff_ds, options=gdal_translate_options)

    # Create PDF with world file
    pdf_options = gdal.TranslateOptions(
        format="PDF",
        outputSRS=projection,
        GCPs=tiff_ds.GetGCPs()
    )
    
    temp_pdf = "temp.pdf"
    gdal.Translate(temp_pdf, temp_tiff, options=pdf_options)

    # Create final PDF with layers using PyMuPDF
    doc = fitz.open()
    pdf_doc = fitz.open(temp_pdf)
    
    # Create new page with same dimensions
    page = doc.new_page(width=pdf_doc[0].rect.width, height=pdf_doc[0].rect.height)
    
    # Add image as layer
    source_page = pdf_doc[0]
    pix = source_page.get_pixmap()
    page.insert_image(page.rect, pixmap=pix)
    
    # Add OCG (Optional Content Group) for layer toggling
    ocg = doc.add_ocg("Georeferenced Image Layer")
    xref = page.get_images()[0][0]  # Get xref of image
    page.set_ocg(xref, ocg)  # Associate image with OCG
    
    # Save final PDF
    doc.save(output_pdf, garbage=4, deflate=True)
    doc.close()
    pdf_doc.close()
    
    # Clean up temporary files
    os.remove(temp_tiff)
    os.remove(temp_pdf)
    
    print(f"Created georeferenced PDF with toggleable layer: {output_pdf}")


    
# Example usage
if __name__ == "__main__":
    input_tiff = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\1627_utm\P2_35A_imagesRGB_orthomosaic.tif"
    output_pdf = "only_image.pdf"  # Replace with desired output PDF path
    create_georeferenced_pdf(input_tiff, output_pdf)

AttributeError: 'Page' object has no attribute 'set_ocg'

In [39]:
from osgeo import gdal

def create_layered_geo_pdf(input_tiff, output_pdf):
    gdal.GetDriverByName('PDF').Register()
    
    translate_options = gdal.TranslateOptions(
        format="PDF",
        creationOptions=[
            "LAYER_NAME=Ortho Image",
            "EXTRA_LAYER_NAME=Base Layer",
            "OFF_LAYERS=Base Layer",
            "EXTRA_STREAM=",
            "GEO_ENCODING=ISO32000"
        ]
    )
    
    gdal.Translate(output_pdf, input_tiff, options=translate_options)

input_tiff = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\1627_utm\P2_35A_imagesRGB_orthomosaic.tif"
output_pdf = "georef_layered.pdf"
create_layered_geo_pdf(input_tiff, output_pdf)

In [106]:
from osgeo import gdal, ogr
import os

def create_geo_pdf_with_overlays(tiff_path, shp_folder, output_pdf):
    # Register drivers
    gdal.AllRegister()
    
    # Create vector PDF driver
    pdf_driver = ogr.GetDriverByName('PDF')
    if pdf_driver is None:
        raise Exception("PDF driver not available")
    
    # Create the PDF dataset
    pdf_ds = pdf_driver.CreateDataSource(
        output_pdf,
        options=['WRITE_LAYERS_AS_REFERENCED=NO', 'DPI=300']
    )
    
    # Add image as a layer
    img_options = [
        'LAYER_NAME=BaseImage',
        'IMAGE_INPUT=' + tiff_path
    ]
    
    img_layer = pdf_ds.CreateLayer('image', None, ogr.wkbNone, img_options)
    
    # Add vector layers
    vector_files = {
        'segments': os.path.join(shp_folder, 'segments.shp'),
        'lines': os.path.join(shp_folder, 'lines.shp'),
        'points': os.path.join(shp_folder, 'points.shp')
    }
    
    for layer_name, shp_path in vector_files.items():
        if os.path.exists(shp_path):
            # Open the shapefile
            shp_ds = ogr.Open(shp_path)
            shp_layer = shp_ds.GetLayer()
            
            # Create new layer in PDF
            layer_options = [
                f'LAYER_NAME={layer_name}',
                'FEATURE_TYPE=FEATURE'
            ]
            
            out_layer = pdf_ds.CreateLayer(
                layer_name,
                shp_layer.GetSpatialRef(),
                shp_layer.GetGeomType(),
                layer_options
            )
            
            # Copy field definitions
            layer_defn = shp_layer.GetLayerDefn()
            for i in range(layer_defn.GetFieldCount()):
                out_layer.CreateField(layer_defn.GetFieldDefn(i))
            
            # Copy features
            for feature in shp_layer:
                out_layer.CreateFeature(feature)
            
            # Clean up
            shp_ds = None
    
    # Close the PDF
    pdf_ds = None
    print(f"Created georeferenced PDF with overlays: {output_pdf}")



if __name__ == "__main__":
    # Your file paths
    input_tiff = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\1627_utm\P2_35A_imagesRGB_orthomosaic.tif"
    vector_dir = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\Results\vectors"
    output_pdf = 'output_georeferenced.pdf'
    
    create_geo_pdf_with_overlays(input_tiff, vector_dir, output_pdf)



Created georeferenced PDF with overlays: output_georeferenced.pdf


In [63]:
from osgeo import gdal

# Set PDF creation options
gdal.SetConfigOption('GDAL_PDF_LAYERS', 'YES')
gdal.SetConfigOption('OGR_PDF_WRITE_GEOM_AS_ATTRIBUTES', 'NO')

# Input paths
input_tiff = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\1627_utm\P2_35A_imagesRGB_orthomosaic.tif"
shapefile_folder = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\Results\vectors"

output_pdf = "output.pdf"

# Create list of layers with shapefiles
vector_layers = [
    "segments.shp",
    "lines.shp",
    "points.shp"
]

# Prepare layer string for GDAL
layers_str = ",".join(vector_layers)

# Create PDF with all layers
ds = gdal.Translate(
    output_pdf, 
    input_tiff,
    format='PDF',
    options=[
        '-co', 'DPI=300',
        '-co', f'OGR_DATASOURCE={shapefile_folder}',  # Here's where you specify the shapefile folder
        '-co', f'OGR_DISPLAY_LAYER_NAMES={layers_str}',
        '-co', 'OGR_DISPLAY_FIELD=NAME'
    ]
)

ds = None
print("PDF created successfully!")

PDF created successfully!


In [64]:
from osgeo import gdal

# Set PDF creation options
gdal.SetConfigOption('GDAL_PDF_LAYERS', 'YES')
gdal.SetConfigOption('OGR_PDF_WRITE_GEOM_AS_ATTRIBUTES', 'NO')
gdal.SetConfigOption('OGR_PDF_SYMBOL_SIZE', '1.0')  # Control point size
gdal.SetConfigOption('GDAL_PDF_DPI', '300')  # Higher DPI for better image quality

# Input paths
input_tiff = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\1627_utm\P2_35A_imagesRGB_orthomosaic.tif"
shapefile_folder = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\Results\vectors"
output_pdf = "output_improved.pdf"

# Create list of layers with shapefiles
vector_layers = [
    "segments.shp",
    "lines.shp",
    "points.shp"
]

# Prepare layer string for GDAL
layers_str = ",".join(vector_layers)

# Create PDF with all layers and improved settings
ds = gdal.Translate(
    output_pdf, 
    input_tiff,
    format='PDF',
    options=[
        '-co', 'DPI=300',
        '-co', 'COMPRESS=JPEG',
        '-co', 'JPEG_QUALITY=95',
        '-co', f'OGR_DATASOURCE={shapefile_folder}',
        '-co', f'OGR_DISPLAY_LAYER_NAMES={layers_str}',
        '-co', 'OGR_DISPLAY_FIELD=NAME',
        '-co', 'OGR_PDF_POINTS_AS_CIRCLES=YES',
        '-co', 'OGR_PDF_POINT_SIZE=1',  # Smaller point size
        '-co', 'MARGIN=0',
        '-co', 'EXTRA_STREAM=LAYERS_ON'
    ]
)

ds = None
print("PDF created successfully with improved settings!")

# To verify georeferencing
input_ds = gdal.Open(input_tiff)
if input_ds:
    geotransform = input_ds.GetGeoTransform()
    if geotransform:
        print("Original georeferencing preserved")
        print("Geotransform:", geotransform)
    input_ds = None

PDF created successfully with improved settings!
Original georeferencing preserved
Geotransform: (621064.1450287445, 0.009489075621044705, 0.0, 5687357.855062823, 0.0, -0.009489075621044705)


In [67]:
from osgeo import gdal

# Set PDF creation options
gdal.SetConfigOption('GDAL_PDF_LAYERS', 'YES')
gdal.SetConfigOption('OGR_PDF_WRITE_GEOM_AS_ATTRIBUTES', 'NO')
gdal.SetConfigOption('OGR_PDF_SYMBOL_SIZE', '1.0')
gdal.SetConfigOption('GDAL_PDF_DPI', '300')

# Input paths
input_tiff = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\1627_utm\P2_35A_imagesRGB_orthomosaic.tif"
shapefile_folder = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\Results\vectors"
output_pdf = "output_all_toggleable.pdf"

# Create list of layers with shapefiles
vector_layers = [
    "segments",
    "lines",
    "points"
]

# Prepare layer string for GDAL
layers_str = ",".join(vector_layers)

# Create PDF with all layers and improved settings
ds = gdal.Translate(
    output_pdf, 
    input_tiff,
    format='PDF',
    options=[
        '-co', 'DPI=300',
        '-co', 'COMPRESS=JPEG',
        '-co', 'JPEG_QUALITY=95',
        '-co', f'OGR_DATASOURCE={shapefile_folder}',
        '-co', f'OGR_DISPLAY_LAYER_NAMES={layers_str}',
        '-co', 'OGR_DISPLAY_FIELD=NAME',
        '-co', 'OGR_PDF_POINT_SIZE=0.5',  # Made points smaller
        '-co', 'MARGIN=0',
        '-co', 'EXTRA_STREAM=LAYERS_ON',
        '-co', 'LAYER_NAME=Orthomosaic',  # This makes the base image toggleable
        '-co', 'GEO_ENCODING=ISO32000'    # Ensures proper georeferencing
    ]
)

ds = None
print("PDF created successfully with all layers toggleable!")

PDF created successfully with all layers toggleable!


In [107]:
from osgeo import gdal
import os
from typing import List, Dict

class GeoTiffToPDF:
    def __init__(self, tiff_path: str, shapefile_folder: str, output_pdf: str):
        self.tiff_path = tiff_path
        self.shapefile_folder = shapefile_folder
        self.output_pdf = output_pdf
        self.vector_layers = ["segments", "lines", "points"]
        
        # Initialize GDAL configurations
        self._set_gdal_configs()
    
    def _set_gdal_configs(self) -> None:
        """Set initial GDAL configuration options"""
        configs = {
            'GDAL_PDF_LAYERS': 'YES',
            'OGR_PDF_WRITE_GEOM_AS_ATTRIBUTES': 'NO',
            'OGR_PDF_SYMBOL_SIZE': '1.0',
            'GDAL_PDF_DPI': '300'
        }
        for key, value in configs.items():
            gdal.SetConfigOption(key, value)
    
    def _get_pdf_options(self) -> List[str]:
        """Define PDF creation options"""
        base_options = [
            '-co', 'DPI=300',
            '-co', 'COMPRESS=JPEG',
            '-co', 'JPEG_QUALITY=95',
            '-co', f'OGR_DATASOURCE={self.shapefile_folder}',
            '-co', f'OGR_DISPLAY_LAYER_NAMES={",".join(self.vector_layers)}',
            '-co', 'OGR_DISPLAY_FIELD=NAME',
            '-co', 'OGR_PDF_POINT_SIZE=0.5',
            '-co', 'MARGIN=0',
            '-co', 'EXTRA_STREAM=LAYERS_ON',
            '-co', 'LAYER_NAME=Orthomosaic',
            '-co', 'GEO_ENCODING=ISO32000'
        ]
        return base_options
    
    def validate_inputs(self) -> bool:
        """Validate input files and folders"""
        if not os.path.exists(self.tiff_path):
            raise FileNotFoundError(f"TIFF file not found: {self.tiff_path}")
        
        if not os.path.exists(self.shapefile_folder):
            raise FileNotFoundError(f"Shapefile folder not found: {self.shapefile_folder}")
            
        # Check if all shapefiles exist
        for layer in self.vector_layers:
            shp_path = os.path.join(self.shapefile_folder, f"{layer}.shp")
            if not os.path.exists(shp_path):
                raise FileNotFoundError(f"Shapefile not found: {shp_path}")
        
        return True
    
    def create_pdf(self) -> bool:
        """Create the georeferenced PDF with all layers"""
        try:
            self.validate_inputs()
            
            # Create PDF using GDAL
            ds = gdal.Translate(
                self.output_pdf,
                self.tiff_path,
                format='PDF',
                options=self._get_pdf_options()
            )
            
            # Clean up
            ds = None
            
            if os.path.exists(self.output_pdf):
                print(f"Successfully created PDF: {self.output_pdf}")
                self._print_layer_info()
                return True
            return False
            
        except Exception as e:
            print(f"Error creating PDF: {str(e)}")
            return False
    
    def _print_layer_info(self) -> None:
        """Print information about the created layers"""
        print("\nPDF Layer Information:")
        print("----------------------")
        print("Base Layer:")
        print("  - Orthomosaic (toggleable)")
        print("\nVector Layers:")
        for layer in self.vector_layers:
            print(f"  - {layer}")

def main():
    # Example usage
    tiff_path = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\1627_utm\P2_35A_imagesRGB_orthomosaic.tif"
    shapefile_folder = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\Results\vectors"
    output_pdf = "output_georeferenced.pdf"
    
    # Create converter instance
    converter = GeoTiffToPDF(tiff_path, shapefile_folder, output_pdf)
    
    # Create PDF
    converter.create_pdf()

if __name__ == "__main__":
    main()

Error creating PDF: Unable to create PDF file output_georeferenced.pdf.



In [108]:
from osgeo import gdal
import os
from typing import List, Dict

class GeoTiffToPDF:
    def __init__(self, tiff_path: str, shapefile_folder: str, output_pdf: str):
        self.tiff_path = tiff_path
        self.shapefile_folder = shapefile_folder
        self.output_pdf = output_pdf
        self.vector_layers = ["segments", "lines", "points"]
        
        # Initialize GDAL configurations
        self._set_gdal_configs()
    
    def _set_gdal_configs(self) -> None:
        """Set initial GDAL configuration options"""
        configs = {
            'GDAL_PDF_LAYERS': 'YES',
            'OGR_PDF_WRITE_GEOM_AS_ATTRIBUTES': 'NO',
            'OGR_PDF_SYMBOL_SIZE': '1.0',
            'GDAL_PDF_DPI': '300'
        }
        for key, value in configs.items():
            gdal.SetConfigOption(key, value)
    
    def _get_pdf_options(self) -> List[str]:
        """Define PDF creation options"""
        base_options = [
            '-co', 'DPI=300',
            '-co', f'OGR_DATASOURCE={self.shapefile_folder}',
            '-co', f'OGR_DISPLAY_LAYER_NAMES={",".join(self.vector_layers)}',
            '-co', 'OGR_DISPLAY_FIELD=NAME',
            '-co', 'OGR_PDF_POINT_SIZE=0.01',
            '-co', 'MARGIN=0',
            '-co', 'EXTRA_STREAM=LAYERS_ON',
            '-co', 'LAYER_NAME=Orthomosaic',
            '-co', 'GEO_ENCODING=ISO32000',
            '-co', 'LAYER_NAME_BASE=0',  # Ensure base layer is on bottom
            '-co', 'LAYER_NAME_VECTOR=YES',  # Ensure vector layers are toggleable
        ]
        return base_options
    
    def validate_inputs(self) -> bool:
        """Validate input files and folders"""
        if not os.path.exists(self.tiff_path):
            raise FileNotFoundError(f"TIFF file not found: {self.tiff_path}")
        
        if not os.path.exists(self.shapefile_folder):
            raise FileNotFoundError(f"Shapefile folder not found: {self.shapefile_folder}")
            
        # Check if all shapefiles exist
        for layer in self.vector_layers:
            shp_path = os.path.join(self.shapefile_folder, f"{layer}.shp")
            if not os.path.exists(shp_path):
                raise FileNotFoundError(f"Shapefile not found: {shp_path}")
        
        return True
    
    def create_pdf(self) -> bool:
        """Create the georeferenced PDF with all layers"""
        try:
            self.validate_inputs()
            
            # Create PDF using GDAL
            ds = gdal.Translate(
                self.output_pdf,
                self.tiff_path,
                format='PDF',
                options=self._get_pdf_options()
            )
            
            # Clean up
            ds = None
            
            if os.path.exists(self.output_pdf):
                print(f"Successfully created PDF: {self.output_pdf}")
                self._print_layer_info()
                return True
            return False
            
        except Exception as e:
            print(f"Error creating PDF: {str(e)}")
            return False
    
    def _print_layer_info(self) -> None:
        """Print information about the created layers"""
        print("\nPDF Layer Information:")
        print("----------------------")
        print("Base Layer:")
        print("  - Orthomosaic (toggleable)")
        print("\nVector Layers:")
        for layer in self.vector_layers:
            print(f"  - {layer}")

def main():
    # Example usage
    tiff_path = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\1627_utm\P2_35A_imagesRGB_orthomosaic.tif"
    shapefile_folder = r'C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\Results\vectors'
    output_pdf = "output_georeferenced.pdf"
    
    # Create converter instance
    converter = GeoTiffToPDF(tiff_path, shapefile_folder, output_pdf)
    
    # Create PDF
    converter.create_pdf()

if __name__ == "__main__":
    main()


Successfully created PDF: output_georeferenced.pdf

PDF Layer Information:
----------------------
Base Layer:
  - Orthomosaic (toggleable)

Vector Layers:
  - segments
  - lines
  - points


In [109]:
import os
from osgeo import gdal, ogr, osr
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

class GeoPDFCreator:
    def __init__(self):
        # Register all GDAL drivers
        gdal.AllRegister()
        
        # Set up styling from TreeVectorViz
        self.health_colors = {'0': '#E3412B', '1': '#FBAA35', '2': '#30C876', '3': '#1E8C4D'}
        self.line_colors = {'0': '#3EBCA1', '1': '#D9D9D9', '2': '#FD3E3E'}
        
    def get_point_color(self, height):
        if height <= 1.5:
            return 'orange'
        if height <= 2.5:
            return 'yellow'
        return 'lime'

    def create_pdf_with_layers(self, tiff_path, shp_folder, output_pdf):
        """Create a georeferenced PDF with toggleable layers"""
        
        # Set up the PDF driver
        driver = gdal.GetDriverByName('PDF')
        if driver is None:
            raise Exception("PDF driver not available")
        
        # Read the TIFF file
        tiff_ds = gdal.Open(tiff_path)
        if tiff_ds is None:
            raise Exception(f"Could not open {tiff_path}")
            
        # Get TIFF properties
        projection = tiff_ds.GetProjection()
        geotransform = tiff_ds.GetGeoTransform()
        
        # Set GDAL configurations
        gdal.SetConfigOption('GDAL_PDF_LAYERS', 'YES')
        gdal.SetConfigOption('OGR_PDF_WRITE_GEOM_AS_ATTRIBUTES', 'NO')
        gdal.SetConfigOption('OGR_PDF_SYMBOL_SIZE', '1.0')
        gdal.SetConfigOption('GDAL_PDF_DPI', '300')
        
        # Define layers string
        layers = ["segments", "lines", "points"]
        layers_str = ",".join(layers)
        
        # Create initial PDF with TIFF
        ds = gdal.Translate(
            output_pdf, 
            tiff_path,
            format='PDF',
            options=[
                '-co', 'DPI=300',
                '-co', 'COMPRESS=JPEG',
                '-co', 'JPEG_QUALITY=95',
                '-co', f'OGR_DATASOURCE={shp_folder}',
                '-co', f'OGR_DISPLAY_LAYER_NAMES={layers_str}',
                '-co', 'OGR_DISPLAY_FIELD=NAME',
                '-co', 'OGR_PDF_POINT_SIZE=0.5',
                '-co', 'MARGIN=0',
                '-co', 'EXTRA_STREAM=LAYERS_ON',
                '-co', 'LAYER_NAME=Orthomosaic',
                '-co', 'GEO_ENCODING=ISO32000',
                '-co', 'EXTRA_IMAGES=LAYER:Orthomosaic'  # Makes the image layer toggleable
            ]
        )
        
        # Create OGR PDF datasource for vector layers
        pdf_driver = ogr.GetDriverByName('PDF')
        vector_pdf = pdf_driver.CreateDataSource(output_pdf, options=['UPDATE=YES'])
        
        # Process each shapefile
        shp_files = {
            'segments': os.path.join(shp_folder, 'segments.shp'),
            'lines': os.path.join(shp_folder, 'lines.shp'),
            'points': os.path.join(shp_folder, 'points.shp')
        }
        
        for layer_name, shp_path in shp_files.items():
            if os.path.exists(shp_path):
                # Open shapefile
                shp_ds = ogr.Open(shp_path)
                if shp_ds is None:
                    print(f"Could not open {shp_path}")
                    continue
                
                layer = shp_ds.GetLayer(0)
                
                # Create layer in PDF
                pdf_layer = vector_pdf.CreateLayer(
                    layer_name,
                    layer.GetSpatialRef(),
                    layer.GetGeomType(),
                    options=[f'LAYER_NAME={layer_name.title()}']
                )
                
                # Copy fields from shapefile to PDF
                layer_defn = layer.GetLayerDefn()
                for i in range(layer_defn.GetFieldCount()):
                    field_defn = layer_defn.GetFieldDefn(i)
                    pdf_layer.CreateField(field_defn)
                
                # Copy features with styling
                for feature in layer:
                    out_feature = ogr.Feature(pdf_layer.GetLayerDefn())
                    out_feature.SetGeometry(feature.GetGeometryRef())
                    
                    # Copy attributes
                    for i in range(layer_defn.GetFieldCount()):
                        out_feature.SetField(i, feature.GetField(i))
                    
                    # Set styling based on layer type and attributes
                    if layer_name == 'segments':
                        class_val = str(feature.GetField('class'))
                        color = self.health_colors.get(class_val, '#000000')
                        style = f'PEN(c:{color},w:1px);BRUSH(fc:{color},bc:{color},op:30%)'
                    elif layer_name == 'lines':
                        class_val = str(feature.GetField('class'))
                        color = self.line_colors.get(class_val, '#000000')
                        style = f'PEN(c:{color},w:2px)'
                    elif layer_name == 'points':
                        height = feature.GetField('height_m')
                        color = self.get_point_color(height)
                        style = f'SYMBOL(c:{color},id:circle,s:8px)'
                    
                    out_feature.SetStyleString(style)
                    pdf_layer.CreateFeature(out_feature)
                
                # Cleanup
                shp_ds = None
        
        # Cleanup
        vector_pdf = None
        ds = None
        tiff_ds = None
        
        print(f"Created georeferenced PDF with layers at: {output_pdf}")

def main():
    creator = GeoPDFCreator()
    
    # Replace these paths with your actual paths
    tiff_path = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\1627_utm\P2_35A_imagesRGB_orthomosaic.tif"
    shapefile_folder = r'C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\Results\vectors'
    output_pdf = "output_georeferenced_n.pdf"
    
    creator.create_pdf_with_layers(tiff_path, shapefile_folder, output_pdf)

if __name__ == "__main__":
    main()

Created georeferenced PDF with layers at: output_georeferenced_n.pdf


In [48]:
import geopandas as gpd
from osgeo import gdal, ogr, osr
import os
import math

class ShapefileToPDF:
    def __init__(self):
        self.health_colors = {'0': '#E3412B', '1': '#FBAA35', '2': '#30C876', '3': '#1E8C4D'}
        self.line_colors = {'0': '#3EBCA1', '1': '#D9D9D9', '2': '#FD3E3E'}
    
    def create_pdf_layers(self, input_dir, output_pdf):
        gdal.SetConfigOption('OGR_PDF_LAYER_CREATION_OPTIONS', 'ON')
        gdal.SetConfigOption('OGR_PDF_INCLUDE_LAYER_NAMES', 'ON')
        
        driver = ogr.GetDriverByName('PDF')
        options = [
            'PDF_LAYER_ORDER=ON',
            'OGR_PDF_WRITE_INFO=ON',
            'DPI=300'
        ]
        pdf_ds = driver.CreateDataSource(output_pdf, options=options)
        
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(32610)
        
        # Process layers
        self.process_segments(input_dir, pdf_ds, srs)
        self.process_lines(input_dir, pdf_ds, srs)
        self.process_points(input_dir, pdf_ds, srs)
        
        # Create label layers
        self.create_line_labels(input_dir, pdf_ds, srs)
        self.create_point_labels(input_dir, pdf_ds, srs)
        
        pdf_ds = None
    
    def process_segments(self, input_dir, pdf_ds, srs):
        segments_layer = pdf_ds.CreateLayer('Segments', srs, ogr.wkbPolygon,
                                          options=['LAYER_NAME=Segments'])
        
        segments_shp = os.path.join(input_dir, 'segments.shp')
        segments_ds = ogr.Open(segments_shp)
        segments_src = segments_ds.GetLayer()
        
        # Create fields
        class_field = ogr.FieldDefn('class', ogr.OFTString)
        segments_layer.CreateField(class_field)
        
        # Copy features
        for feature in segments_src:
            out_feature = ogr.Feature(segments_layer.GetLayerDefn())
            out_feature.SetGeometry(feature.GetGeometryRef().Clone())
            class_val = feature.GetField('class')
            out_feature.SetField('class', str(class_val))
            out_feature.SetStyleString(f'BRUSH(fc:{self.health_colors[str(class_val)]},bc:#000000);PEN(c:#000000,w:0.1pt)')
            segments_layer.CreateFeature(out_feature)
        
        segments_ds = None
    
    def process_lines(self, input_dir, pdf_ds, srs):
        lines_layer = pdf_ds.CreateLayer('Lines', srs, ogr.wkbLineString,
                                       options=['LAYER_NAME=Lines'])
        
        lines_shp = os.path.join(input_dir, 'lines.shp')
        lines_ds = ogr.Open(lines_shp)
        lines_src = lines_ds.GetLayer()
        
        # Create fields
        class_field = ogr.FieldDefn('class', ogr.OFTString)
        distance_field = ogr.FieldDefn('distance', ogr.OFTReal)
        lines_layer.CreateField(class_field)
        lines_layer.CreateField(distance_field)
        
        # Copy features
        for feature in lines_src:
            out_feature = ogr.Feature(lines_layer.GetLayerDefn())
            out_feature.SetGeometry(feature.GetGeometryRef().Clone())
            class_val = str(feature.GetField('class'))
            distance = feature.GetField('distance')
            out_feature.SetField('class', class_val)
            out_feature.SetField('distance', distance)
            out_feature.SetStyleString(f'PEN(c:{self.line_colors[class_val]},w:2pt)')
            lines_layer.CreateFeature(out_feature)
        
        lines_ds = None
    
    def process_points(self, input_dir, pdf_ds, srs):
        points_layer = pdf_ds.CreateLayer('Points', srs, ogr.wkbPoint,
                                        options=['LAYER_NAME=Points'])
        
        points_shp = os.path.join(input_dir, 'points.shp')
        points_ds = ogr.Open(points_shp)
        points_src = points_ds.GetLayer()
        
        # Create fields
        class_field = ogr.FieldDefn('class', ogr.OFTString)
        height_field = ogr.FieldDefn('height_m', ogr.OFTReal)
        points_layer.CreateField(class_field)
        points_layer.CreateField(height_field)
        
        # Copy features
        for feature in points_src:
            out_feature = ogr.Feature(points_layer.GetLayerDefn())
            out_feature.SetGeometry(feature.GetGeometryRef().Clone())
            height = feature.GetField('height_m')
            class_val = feature.GetField('class')
            
            # Get color from height since pointCol might not be available
            if height <= 1.5:
                point_color = '#FFA500'  # orange
            elif height <= 2.5:
                point_color = '#00FF00'  # lime green
            else:
                point_color = '#32CD32'  # darker green
            
            out_feature.SetField('class', str(class_val))
            out_feature.SetField('height_m', height)
            
            # Style points based on class
            if class_val == '1':
                # For class 1: colored circle with black inner circle
                style = f'SYMBOL(id:circle,c:{point_color},s:2.5pt);SYMBOL(id:circle,c:#000000,s:1pt)'
            else:
                # For class 0: just colored circle
                style = f'SYMBOL(id:circle,c:{point_color},s:2.5pt)'
                
            out_feature.SetStyleString(style)
            points_layer.CreateFeature(out_feature)
        
        points_ds = None
    
    def create_line_labels(self, input_dir, pdf_ds, srs):
        label_layer = pdf_ds.CreateLayer('Line_Labels', srs, ogr.wkbPoint,
                                       options=['LAYER_NAME=Line Distances'])
        
        lines_shp = os.path.join(input_dir, 'lines.shp')
        lines_ds = ogr.Open(lines_shp)
        lines_src = lines_ds.GetLayer()
        
        label_field = ogr.FieldDefn('label_text', ogr.OFTString)
        label_layer.CreateField(label_field)
        
        for feature in lines_src:
            geom = feature.GetGeometryRef()
            distance = feature.GetField('distance')
            
            # Get points to calculate angle
            points = geom.GetPoints()
            if len(points) >= 2:
                start_point = points[0]
                end_point = points[-1]
                
                # Calculate midpoint
                mid_x = (start_point[0] + end_point[0]) / 2
                mid_y = (start_point[1] + end_point[1]) / 2
                
                # Calculate angle in degrees
                dx = end_point[0] - start_point[0]
                dy = end_point[1] - start_point[1]
                angle = math.degrees(math.atan2(dy, dx))
                
                # Adjust angle to keep text readable (not upside down)
                if angle < -90 or angle > 90:
                    angle += 180
                
                # Create point geometry for label
                point = ogr.Geometry(ogr.wkbPoint)
                point.AddPoint(mid_x, mid_y)
                
                out_feature = ogr.Feature(label_layer.GetLayerDefn())
                out_feature.SetGeometry(point)
                label_text = f'{distance:.1f}m'
                out_feature.SetField('label_text', label_text)
                
                # Add angle to label style
                style = f'LABEL(f:"Arial",s:10pt,t:{label_text},c:#000000,a:{angle:.1f})'
                out_feature.SetStyleString(style)
                label_layer.CreateFeature(out_feature)
        
        lines_ds = None
    
    def create_point_labels(self, input_dir, pdf_ds, srs):
        label_layer = pdf_ds.CreateLayer('Point_Labels', srs, ogr.wkbPoint,
                                       options=['LAYER_NAME=Point Heights'])
        
        points_shp = os.path.join(input_dir, 'points.shp')
        points_ds = ogr.Open(points_shp)
        points_src = points_ds.GetLayer()
        
        label_field = ogr.FieldDefn('label_text', ogr.OFTString)
        label_layer.CreateField(label_field)
        
        for feature in points_src:
            out_feature = ogr.Feature(label_layer.GetLayerDefn())
            out_feature.SetGeometry(feature.GetGeometryRef().Clone())
            height = feature.GetField('height_m')
            # Enhanced label style with bold text and background halo
            label_text = f'{height:.1f}m'
            out_feature.SetField('label_text', label_text)
            out_feature.SetStyleString(f'LABEL(f:"Arial Bold",s:8pt,t:{label_text},c:#000000,dx:6,dy:3,bo:#FFFFFF,hc:#FFFFFF,ho:2.5)')
            label_layer.CreateFeature(out_feature)
        
        points_ds = None

if __name__ == "__main__":
    converter = ShapefileToPDF()
    input_directory = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\Results\vectors"
    output_pdf = "output_layers_noor.pdf"
    converter.create_pdf_layers(input_directory, output_pdf)

In [6]:
import geopandas as gpd
from osgeo import gdal, ogr, osr
import os
import math

class ShapefileToPDF:
    def __init__(self):
        self.health_colors = {'0': '#E3412B', '1': '#FBAA35', '2': '#30C876', '3': '#1E8C4D'}
        self.line_colors = {'0': '#3EBCA1', '1': '#D9D9D9', '2': '#FD3E3E'}
    
    def get_point_color(self, height):
        if height <= 1.5:
            return 'orange'
        if height <= 2.5:
            return 'yellow'
        return 'lime'
    
    def add_raster_layer(self, input_raster, pdf_ds, srs):
        """
        Add a raster image as the base layer in the PDF.
        
        Parameters:
        input_raster (str): Path to the input raster file (e.g., GeoTIFF)
        pdf_ds: PDF datasource object
        srs: Spatial reference system
        """
        # Open the raster dataset
        raster_ds = gdal.Open(input_raster)
        if raster_ds is None:
            raise Exception(f"Failed to open raster file: {input_raster}")
            
        # Create raster layer with specific options for base layer
        raster_options = [
            'LAYER_NAME=Base Image',
            'LAYER_TYPE=RASTER',
            'OPACITY=100',
            'LAYER_ORDER=BOTTOM'  # This ensures the raster is placed at the bottom
        ]
        
        # Create the raster layer in the PDF
        pdf_layer = pdf_ds.CreateLayer('Base_Image', srs, ogr.wkbUnknown, options=raster_options)
        
        # Create a memory dataset to hold the raster data
        mem_driver = gdal.GetDriverByName('MEM')
        mem_ds = mem_driver.CreateCopy('', raster_ds)
        
        # Add the raster to the PDF layer
        gdal.Layer.CreateCopy(pdf_layer, mem_ds)
        
        # Clean up
        mem_ds = None
        raster_ds = None
    
    def create_pdf_layers(self, input_dir, output_pdf, base_image=None):
        gdal.SetConfigOption('OGR_PDF_LAYER_CREATION_OPTIONS', 'ON')
        gdal.SetConfigOption('OGR_PDF_INCLUDE_LAYER_NAMES', 'ON')
        
        driver = ogr.GetDriverByName('PDF')
        options = [
            'PDF_LAYER_ORDER=ON',
            'OGR_PDF_WRITE_INFO=ON',
            'DPI=300',
            'LAYER_OPTIONS=ON'  # Enable layer options including visibility
        ]
        pdf_ds = driver.CreateDataSource(output_pdf, options=options)
        
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(32610)
        
        # Add base image layer if provided
        if base_image and os.path.exists(base_image):
            self.add_raster_layer(base_image, pdf_ds, srs)
        
        # Process vector layers
        self.process_segments(input_dir, pdf_ds, srs)
        self.process_lines(input_dir, pdf_ds, srs)
        self.process_points(input_dir, pdf_ds, srs)
        
        # Create label layers
        self.create_line_labels(input_dir, pdf_ds, srs)
        self.create_point_labels(input_dir, pdf_ds, srs)
        
        pdf_ds = None
    
    def process_segments(self, input_dir, pdf_ds, srs):
        segments_layer = pdf_ds.CreateLayer('Segments', srs, ogr.wkbPolygon,
                                          options=['LAYER_NAME=Segments'])
        
        segments_shp = os.path.join(input_dir, 'segments.shp')
        segments_ds = ogr.Open(segments_shp)
        segments_src = segments_ds.GetLayer()
        
        # Create fields
        class_field = ogr.FieldDefn('class', ogr.OFTString)
        segments_layer.CreateField(class_field)
        
        # Copy features
        for feature in segments_src:
            out_feature = ogr.Feature(segments_layer.GetLayerDefn())
            out_feature.SetGeometry(feature.GetGeometryRef().Clone())
            class_val = feature.GetField('class')
            out_feature.SetField('class', str(class_val))
            out_feature.SetStyleString(f'BRUSH(fc:{self.health_colors[str(class_val)]},bc:#000000);PEN(c:#000000,w:0.1pt)')
            segments_layer.CreateFeature(out_feature)
        
        segments_ds = None
    
    def process_lines(self, input_dir, pdf_ds, srs):
        lines_layer = pdf_ds.CreateLayer('Lines', srs, ogr.wkbLineString,
                                       options=['LAYER_NAME=Lines'])
        
        lines_shp = os.path.join(input_dir, 'lines.shp')
        lines_ds = ogr.Open(lines_shp)
        lines_src = lines_ds.GetLayer()
        
        # Create fields
        class_field = ogr.FieldDefn('class', ogr.OFTString)
        distance_field = ogr.FieldDefn('distance', ogr.OFTReal)
        lines_layer.CreateField(class_field)
        lines_layer.CreateField(distance_field)
        
        # Copy features
        for feature in lines_src:
            out_feature = ogr.Feature(lines_layer.GetLayerDefn())
            out_feature.SetGeometry(feature.GetGeometryRef().Clone())
            class_val = str(feature.GetField('class'))
            distance = feature.GetField('distance')
            out_feature.SetField('class', class_val)
            out_feature.SetField('distance', distance)
            out_feature.SetStyleString(f'PEN(c:{self.line_colors[class_val]},w:2pt)')
            lines_layer.CreateFeature(out_feature)
        
        lines_ds = None
    
    def process_points(self, input_dir, pdf_ds, srs):
        points_layer = pdf_ds.CreateLayer('Points', srs, ogr.wkbPoint,
                                        options=['LAYER_NAME=Points'])
        
        points_shp = os.path.join(input_dir, 'points.shp')
        points_ds = ogr.Open(points_shp)
        points_src = points_ds.GetLayer()
        
        # Create fields
        class_field = ogr.FieldDefn('class', ogr.OFTString)
        height_field = ogr.FieldDefn('height_m', ogr.OFTReal)
        points_layer.CreateField(class_field)
        points_layer.CreateField(height_field)
        
        # Copy features
        for feature in points_src:
            out_feature = ogr.Feature(points_layer.GetLayerDefn())
            out_feature.SetGeometry(feature.GetGeometryRef().Clone())
            height = feature.GetField('height_m')
            class_val = feature.GetField('class')
            
            # Get color from height since pointCol might not be available
            if height <= 1.5:
                point_color = '#FFA500'  # orange
            elif height <= 2.5:
                point_color = '#00FF00'  # lime green
            else:
                point_color = '#32CD32'  # darker green
            
            out_feature.SetField('class', str(class_val))
            out_feature.SetField('height_m', height)
            
            # Style points based on class
            if class_val == '1':
                # For class 1: colored circle with black inner circle
                style = f'SYMBOL(id:circle,c:{point_color},s:2.5pt);SYMBOL(id:circle,c:#000000,s:1pt)'
            else:
                # For class 0: just colored circle
                style = f'SYMBOL(id:circle,c:{point_color},s:2.5pt)'
                
            out_feature.SetStyleString(style)
            points_layer.CreateFeature(out_feature)
        
        points_ds = None
    
    
    def create_line_labels(self, input_dir, pdf_ds, srs):
        label_layer = pdf_ds.CreateLayer('Line_Labels', srs, ogr.wkbPoint,
                                       options=['LAYER_NAME=Line Distances'])
        
        lines_shp = os.path.join(input_dir, 'lines.shp')
        lines_ds = ogr.Open(lines_shp)
        lines_src = lines_ds.GetLayer()
        
        label_field = ogr.FieldDefn('label_text', ogr.OFTString)
        label_layer.CreateField(label_field)
        
        for feature in lines_src:
            geom = feature.GetGeometryRef()
            distance = feature.GetField('distance')
            
            # Get points to calculate angle
            points = geom.GetPoints()
            if len(points) >= 2:
                start_point = points[0]
                end_point = points[-1]
                
                # Calculate midpoint
                mid_x = (start_point[0] + end_point[0]) / 2
                mid_y = (start_point[1] + end_point[1]) / 2
                
                # Calculate angle in degrees
                dx = end_point[0] - start_point[0]
                dy = end_point[1] - start_point[1]
                angle = math.degrees(math.atan2(dy, dx))
                
                # Adjust angle to keep text readable (not upside down)
                if angle < -90 or angle > 90:
                    angle += 180
                
                # Create point geometry for label
                point = ogr.Geometry(ogr.wkbPoint)
                point.AddPoint(mid_x, mid_y)
                
                out_feature = ogr.Feature(label_layer.GetLayerDefn())
                out_feature.SetGeometry(point)
                label_text = f'{distance:.1f}m'
                out_feature.SetField('label_text', label_text)
                
                # Add angle to label style
                style = f'LABEL(f:"Arial",s:10pt,t:{label_text},c:#000000,a:{angle:.1f})'
                out_feature.SetStyleString(style)
                label_layer.CreateFeature(out_feature)
        
        lines_ds = None
    
    def create_point_labels(self, input_dir, pdf_ds, srs):
        label_layer = pdf_ds.CreateLayer('Point_Labels', srs, ogr.wkbPoint,
                                       options=['LAYER_NAME=Point Heights'])
        
        points_shp = os.path.join(input_dir, 'points.shp')
        points_ds = ogr.Open(points_shp)
        points_src = points_ds.GetLayer()
        
        label_field = ogr.FieldDefn('label_text', ogr.OFTString)
        label_layer.CreateField(label_field)
        
        for feature in points_src:
            out_feature = ogr.Feature(label_layer.GetLayerDefn())
            out_feature.SetGeometry(feature.GetGeometryRef().Clone())
            height = feature.GetField('height_m')
            # Enhanced label style with bold text and background halo
            label_text = f'{height:.1f}m'
            out_feature.SetField('label_text', label_text)
            out_feature.SetStyleString(f'LABEL(f:"Arial Bold",s:8pt,t:{label_text},c:#000000,dx:6,dy:3,bo:#FFFFFF,hc:#FFFFFF,ho:2.5)')
            label_layer.CreateFeature(out_feature)
        
        points_ds = None

if __name__ == "__main__":
    converter = ShapefileToPDF()
    input_directory = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\Results\vectors"
    output_pdf = "output_layers_M.pdf"
    converter.create_pdf_layers(input_directory, output_pdf)

In [62]:
def process_tiff_layer(self, input_tiff, pdf_ds, srs):
    """
    Debug version of TIFF processing
    """
    print(f"\nDEBUG: Starting TIFF processing")
    print(f"Input TIFF: {input_tiff}")
    
    # Check TIFF file
    tiff_ds = gdal.Open(input_tiff)
    if tiff_ds is None:
        print("ERROR: Failed to open TIFF file")
        return
    
    print("\nTIFF Properties:")
    print(f"Raster Size: {tiff_ds.RasterXSize} x {tiff_ds.RasterYSize}")
    print(f"Projection: {tiff_ds.GetProjection()}")
    print(f"GeoTransform: {tiff_ds.GetGeoTransform()}")
    
    # Create raster layer
    tiff_layer = pdf_ds.CreateLayer('Base_Imagery', srs, ogr.wkbUnknown,
                                   options=['LAYER_NAME=Base Imagery'])
    print("\nLayer created successfully")
    
    # Get geotransform
    gt = tiff_ds.GetGeoTransform()
    
    # Calculate extent
    width = tiff_ds.RasterXSize
    height = tiff_ds.RasterYSize
    minx = gt[0]
    maxx = gt[0] + gt[1] * width
    miny = gt[3] + gt[5] * height
    maxy = gt[3]
    
    print(f"\nCalculated Extent:")
    print(f"MinX: {minx}, MaxX: {maxx}")
    print(f"MinY: {miny}, MaxY: {maxy}")
    
    # Create polygon for extent
    ring = ogr.Geometry(ogr.wkbLinearRing)
    ring.AddPoint(minx, miny)
    ring.AddPoint(maxx, miny)
    ring.AddPoint(maxx, maxy)
    ring.AddPoint(minx, maxy)
    ring.AddPoint(minx, miny)
    
    poly = ogr.Geometry(ogr.wkbPolygon)
    poly.AddGeometry(ring)
    
    print("\nGeometry created successfully")
    
    # Create feature
    feature = ogr.Feature(tiff_layer.GetLayerDefn())
    feature.SetGeometry(poly)
    
    # Set raster style
    style = (
        f'RASTER({input_tiff},'
        f'WORLDFILE=YES,'
        f'ALPHA=75)'
    )
    print(f"\nStyle string: {style}")
    
    feature.SetStyleString(style)
    result = tiff_layer.CreateFeature(feature)
    print(f"Feature creation result: {result}")
    
    tiff_ds = None
    return tiff_layer

def create_pdf_layers(self, input_dir, output_pdf, tiff_path=None):
    """
    Debug version of PDF creation
    """
    print("\nDEBUG: Starting PDF creation")
    
    gdal.SetConfigOption('OGR_PDF_LAYER_CREATION_OPTIONS', 'ON')
    gdal.SetConfigOption('OGR_PDF_INCLUDE_LAYER_NAMES', 'ON')
    gdal.SetConfigOption('GDAL_PDF_BANDS', '4')
    gdal.SetConfigOption('OGR_PDF_GEO_ENCODING', 'ISO32000')
    
    # Get TIFF info first if available
    bbox_options = []
    if tiff_path:
        print(f"\nProcessing TIFF: {tiff_path}")
        tiff_ds = gdal.Open(tiff_path)
        if tiff_ds:
            gt = tiff_ds.GetGeoTransform()
            print(f"TIFF GeoTransform: {gt}")
            width = tiff_ds.RasterXSize
            height = tiff_ds.RasterYSize
            
            minx = gt[0]
            maxx = gt[0] + gt[1] * width
            miny = gt[3] + gt[5] * height
            maxy = gt[3]
            
            bbox_options = [
                f'BBOX={minx},{miny},{maxx},{maxy}',
                f'IMAGE_EXTENT={minx},{miny},{maxx},{maxy}'
            ]
            print(f"Created bbox options: {bbox_options}")
            tiff_ds = None
    
    # Create PDF with options
    driver = ogr.GetDriverByName('PDF')
    options = [
        'PDF_LAYER_ORDER=ON',
        'OGR_PDF_WRITE_INFO=ON',
        'DPI=300',
        'EXTRA_STREAM=OPACITY:1',
        'COMPOSITION=AUTO',
        'MARGIN=0',
        'GEO_ENCODING=ISO32000',
        'NEATLINE=NO'
    ]
    options.extend(bbox_options)
    
    print(f"\nCreating PDF with options: {options}")
    pdf_ds = driver.CreateDataSource(output_pdf, options=options)
    
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(32610)
    
    # Process TIFF first
    if tiff_path:
        try:
            print("\nAdding TIFF layer...")
            self.process_tiff_layer(tiff_path, pdf_ds, srs)
        except Exception as e:
            print(f"ERROR processing TIFF: {str(e)}")
            import traceback
            print(traceback.format_exc())
    
    # Process other layers
    print("\nProcessing vector layers...")
    self.process_segments(input_dir, pdf_ds, srs)
    self.process_lines(input_dir, pdf_ds, srs)
    self.process_points(input_dir, pdf_ds, srs)
    self.process_toggle_points(input_dir, pdf_ds, srs)
    
    # Create label layers
    self.create_line_labels(input_dir, pdf_ds, srs)
    self.create_point_labels(input_dir, pdf_ds, srs)
    
    pdf_ds = None
    print("\nPDF creation completed")

AttributeError: 'ShapefileToPDF' object has no attribute 'create_pdf_layers'

In [65]:
import geopandas as gpd
from osgeo import gdal, ogr, osr
import os
import math

class ShapefileToPDF:
    def __init__(self):
        self.health_colors = {'0': '#E3412B', '1': '#FBAA35', '2': '#30C876', '3': '#1E8C4D'}
        self.line_colors = {'0': '#3EBCA1', '1': '#D9D9D9', '2': '#FD3E3E'}
    
    def get_point_color(self, height):
        if height <= 1.5:
            return 'orange'
        if height <= 2.5:
            return 'yellow'
        return 'lime'
    
    def create_pdf_layers(self, input_dir, output_pdf):
        gdal.SetConfigOption('OGR_PDF_LAYER_CREATION_OPTIONS', 'ON')
        gdal.SetConfigOption('OGR_PDF_INCLUDE_LAYER_NAMES', 'ON')
        
        driver = ogr.GetDriverByName('PDF')
        options = [
            'PDF_LAYER_ORDER=ON',
            'OGR_PDF_WRITE_INFO=ON',
            'DPI=300'
        ]
        pdf_ds = driver.CreateDataSource(output_pdf, options=options)
        
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(32610)
        
        # Process layers
        self.process_segments(input_dir, pdf_ds, srs)
        self.process_lines(input_dir, pdf_ds, srs)
        self.process_points(input_dir, pdf_ds, srs)
        
        # Create label layers
        self.create_line_labels(input_dir, pdf_ds, srs)
        self.create_point_labels(input_dir, pdf_ds, srs)
        
        pdf_ds = None
    
    def process_segments(self, input_dir, pdf_ds, srs):
        segments_layer = pdf_ds.CreateLayer('Segments', srs, ogr.wkbPolygon,
                                          options=['LAYER_NAME=Segments'])
        
        segments_shp = os.path.join(input_dir, 'segments.shp')
        segments_ds = ogr.Open(segments_shp)
        segments_src = segments_ds.GetLayer()
        
        # Create fields
        class_field = ogr.FieldDefn('class', ogr.OFTString)
        segments_layer.CreateField(class_field)
        
        # Copy features
        for feature in segments_src:
            out_feature = ogr.Feature(segments_layer.GetLayerDefn())
            out_feature.SetGeometry(feature.GetGeometryRef().Clone())
            class_val = feature.GetField('class')
            out_feature.SetField('class', str(class_val))
            out_feature.SetStyleString(f'BRUSH(fc:{self.health_colors[str(class_val)]},bc:#000000);PEN(c:#000000,w:0.1pt)')
            segments_layer.CreateFeature(out_feature)
        
        segments_ds = None
    
    def process_lines(self, input_dir, pdf_ds, srs):
        lines_layer = pdf_ds.CreateLayer('Lines', srs, ogr.wkbLineString,
                                       options=['LAYER_NAME=Lines'])
        
        lines_shp = os.path.join(input_dir, 'lines.shp')
        lines_ds = ogr.Open(lines_shp)
        lines_src = lines_ds.GetLayer()
        
        # Create fields
        class_field = ogr.FieldDefn('class', ogr.OFTString)
        distance_field = ogr.FieldDefn('distance', ogr.OFTReal)
        lines_layer.CreateField(class_field)
        lines_layer.CreateField(distance_field)
        
        # Copy features
        for feature in lines_src:
            out_feature = ogr.Feature(lines_layer.GetLayerDefn())
            out_feature.SetGeometry(feature.GetGeometryRef().Clone())
            class_val = str(feature.GetField('class'))
            distance = feature.GetField('distance')
            out_feature.SetField('class', class_val)
            out_feature.SetField('distance', distance)
            out_feature.SetStyleString(f'PEN(c:{self.line_colors[class_val]},w:2pt)')
            lines_layer.CreateFeature(out_feature)
        
        lines_ds = None
    

    def process_points(self, input_dir, pdf_ds, srs):
        points_layer = pdf_ds.CreateLayer('Points', srs, ogr.wkbPoint,
                                        options=['LAYER_NAME=Points'])
        
        points_shp = os.path.join(input_dir, 'points.shp')
        points_ds = ogr.Open(points_shp)
        points_src = points_ds.GetLayer()
        
        # Create fields
        class_field = ogr.FieldDefn('class', ogr.OFTString)
        height_field = ogr.FieldDefn('height_m', ogr.OFTReal)
        points_layer.CreateField(class_field)
        points_layer.CreateField(height_field)
        
        # Copy features
        for feature in points_src:
            out_feature = ogr.Feature(points_layer.GetLayerDefn())
            out_feature.SetGeometry(feature.GetGeometryRef().Clone())
            height = feature.GetField('height_m')
            class_val = feature.GetField('class')
            
            # Get color from height
            if height <= 1.5:
                point_color = '#FFA500'  # orange
            elif height <= 2.5:
                point_color = '#00FF00'  # lime green
            else:
                point_color = '#32CD32'  # darker green
            
            out_feature.SetField('class', str(class_val))
            out_feature.SetField('height_m', height)
            
            # Style points with white border
            # Order matters: first draw larger white circle, then smaller colored circle
            style = (
                f'SYMBOL(id:circle,c:#FFFFFF,s:3.5pt);'  # White border (larger circle)
                f'SYMBOL(id:circle,c:{point_color},s:2.5pt)'  # Inner colored circle (smaller)
            )
            out_feature.SetStyleString(style)
            points_layer.CreateFeature(out_feature)
        
        points_ds = None
        
         
    def create_line_labels(self, input_dir, pdf_ds, srs):
        label_layer = pdf_ds.CreateLayer('Line_Labels', srs, ogr.wkbPoint,
                                       options=['LAYER_NAME=Line Distances'])
        
        lines_shp = os.path.join(input_dir, 'lines.shp')
        lines_ds = ogr.Open(lines_shp)
        lines_src = lines_ds.GetLayer()
        
        label_field = ogr.FieldDefn('label_text', ogr.OFTString)
        label_layer.CreateField(label_field)
        
        for feature in lines_src:
            geom = feature.GetGeometryRef()
            distance = feature.GetField('distance')
            
            # Get points to calculate angle
            points = geom.GetPoints()
            if len(points) >= 2:
                start_point = points[0]
                end_point = points[-1]
                
                # Calculate midpoint
                mid_x = (start_point[0] + end_point[0]) / 2
                mid_y = (start_point[1] + end_point[1]) / 2
                
                # Calculate angle in degrees
                dx = end_point[0] - start_point[0]
                dy = end_point[1] - start_point[1]
                angle = math.degrees(math.atan2(dy, dx))
                
                # Adjust angle to keep text readable (not upside down)
                if angle < -90 or angle > 90:
                    angle += 180
                
                # Create point geometry for label
                point = ogr.Geometry(ogr.wkbPoint)
                point.AddPoint(mid_x, mid_y)
                
                out_feature = ogr.Feature(label_layer.GetLayerDefn())
                out_feature.SetGeometry(point)
                label_text = f'{distance:.1f}m'
                out_feature.SetField('label_text', label_text)
                
                # Add angle to label style
                style = f'LABEL(f:"Arial",s:10pt,t:{label_text},c:#000000,a:{angle:.1f})'
                out_feature.SetStyleString(style)
                label_layer.CreateFeature(out_feature)
        
        lines_ds = None
    
    def create_point_labels(self, input_dir, pdf_ds, srs):
        label_layer = pdf_ds.CreateLayer('Point_Labels', srs, ogr.wkbPoint,
                                       options=['LAYER_NAME=Point Heights'])
        
        points_shp = os.path.join(input_dir, 'points.shp')
        points_ds = ogr.Open(points_shp)
        points_src = points_ds.GetLayer()
        
        label_field = ogr.FieldDefn('label_text', ogr.OFTString)
        label_layer.CreateField(label_field)
        
        for feature in points_src:
            out_feature = ogr.Feature(label_layer.GetLayerDefn())
            out_feature.SetGeometry(feature.GetGeometryRef().Clone())
            height = feature.GetField('height_m')
            # Enhanced label style with bold text and background halo
            label_text = f'{height:.1f}m'
            out_feature.SetField('label_text', label_text)
            out_feature.SetStyleString(f'LABEL(f:"Arial Bold",s:8pt,t:{label_text},c:#000000,dx:6,dy:3,bo:#FFFFFF,hc:#FFFFFF,ho:2.5)')
            label_layer.CreateFeature(out_feature)
        
        points_ds = None

if __name__ == "__main__":
    converter = ShapefileToPDF()
    input_directory = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\Results\vectors"
    output_pdf = "output_layers_noor.pdf"
    converter.create_pdf_layers(input_directory, output_pdf)

In [73]:
from osgeo import gdal
import os

def merge_pdfs_alternative(vector_pdf, image_pdf, output_pdf):
    """
    Alternative method to merge PDFs using GDAL
    """
    # Configure GDAL
    gdal.SetConfigOption('GDAL_PDF_LAYERS', 'ON')
    
    # First create a copy of the image PDF
    image_options = gdal.TranslateOptions(
        format="PDF",
        creationOptions=[
            "LAYER_NAME=Base Image",
            "GEO_ENCODING=ISO32000",
            "DPI=300"
        ]
    )
    
    try:
        # Create temporary file for intermediate step
        temp_pdf = "temp_merged.pdf"
        
        # First copy the image PDF
        gdal.Translate(temp_pdf, image_pdf, options=image_options)
        
        # Now append the vector PDF
        vector_options = gdal.TranslateOptions(
            format="PDF",
            creationOptions=[
                "APPEND=YES",
                "LAYER_NAME=Vector Data",
                "GEO_ENCODING=ISO32000",
                "EXTRA_STREAM=OPACITY:1"
            ]
        )
        
        # Append vector data
        gdal.Translate(output_pdf, vector_pdf, options=vector_options)
        
        print("PDFs merged successfully!")
        return True
        
    except Exception as e:
        print(f"Error during merge: {str(e)}")
        return False
        
    finally:
        # Clean up temporary file
        if os.path.exists(temp_pdf):
            os.remove(temp_pdf)

# Example usage
if __name__ == "__main__":
    vector_pdf_path = r"C:\Users\User\Desktop\Ronit-Projects\Ronit-Yolo\Forest-Pipeline\output_layers_noor.pdf"
    image_pdf_path = r"C:\Users\User\Desktop\Ronit-Projects\Ronit-Yolo\Forest-Pipeline\temp_base.pdf"
    output_pdf_path = "merged_output.pdf"
    
    merge_pdfs_alternative(vector_pdf_path, image_pdf_path, output_pdf_path)

Error during merge: Received a NULL pointer.


In [3]:
import geopandas as gpd
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import fitz
import io
import numpy as np

def create_layered_pdf(tiff_path, shapefile_paths, output_pdf):
    # Create a new PDF document
    doc = fitz.Document()
    page = doc.new_page()
    
    # First create the base TIFF layer
    with rasterio.open(tiff_path) as src:
        tiff_data = src.read()
        tiff_transform = src.transform
        
        fig, ax = plt.subplots(figsize=(11, 8.5))
        show(tiff_data, transform=tiff_transform, ax=ax)
        
        # Save figure to bytes
        img_bytes = io.BytesIO()
        plt.savefig(img_bytes, format='png', dpi=300)
        plt.close()
        
        # Create image layer
        img_bytes.seek(0)
        img = fitz.Pixmap(img_bytes.read())
        page.insert_image(page.rect, pixmap=img)
        
        # Create OCG (Optional Content Group) for the image
        doc.add_ocg("Base Image")
        
    # Create layers for each shapefile
    for shp_path in shapefile_paths:
        # Read shapefile
        gdf = gpd.read_file(shp_path)
        
        # Create new figure with transparent background
        fig, ax = plt.subplots(figsize=(11, 8.5))
        ax.set_axis_off()
        gdf.plot(ax=ax, alpha=0.5)
        
        # Save to bytes
        shape_bytes = io.BytesIO()
        plt.savefig(shape_bytes, format='png', transparent=True, dpi=300)
        plt.close()
        
        # Create layer
        shape_bytes.seek(0)
        shape_img = fitz.Pixmap(shape_bytes.read())
        
        # Add as new OCG layer
        layer_name = f"Layer_{shp_path.split('/')[-1]}"
        doc.add_ocg(layer_name)
        page.show_pdf_page(page.rect, doc, 0, oc=layer_name)
        
    # Save the PDF
    doc.save(output_pdf)
    doc.close()



# Usage
tiff_path = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\1627_utm\P2_35A_imagesRGB_orthomosaic.tif"
shapefile_paths = [r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\Results\vectors\segments.shp"]
output_pdf = 'output_map.pdf'

create_layered_pdf(tiff_path, shapefile_paths, output_pdf)

AttributeError: module 'fitz' has no attribute 'Document'

In [7]:
from osgeo import gdal, ogr, osr
import os

def modify_shapefile_style(input_dir):
    """Modify shapefile to include style attributes"""
    health_colors = {'0': '#E3412B', '1': '#FBAA35', '2': '#30C876', '3': '#1E8C4D'}
    
    # Open the shapefile
    segments_shp = os.path.join(input_dir, 'segments.shp')
    driver = ogr.GetDriverByName('ESRI Shapefile')
    ds = driver.Open(segments_shp, 1)  # 1 means write mode
    layer = ds.GetLayer()
    
    # Add style field if it doesn't exist
    style_field = ogr.FieldDefn('OGR_STYLE', ogr.OFTString)
    try:
        layer.CreateField(style_field)
    except:
        pass  # Field might already exist
    
    # Update features with style
    for feature in layer:
        class_val = str(feature.GetField('class'))
        if class_val in health_colors:
            color = health_colors[class_val]
            style_string = f'BRUSH(fc:{color});PEN(c:#000000,w:0.1pt)'
            feature.SetField('OGR_STYLE', style_string)
            layer.SetFeature(feature)
    
    # Close the shapefile
    ds = None

def create_georef_pdf(input_tiff, input_dir, output_pdf):
    """Create georeferenced PDF with TIFF and vector layers"""
    # First modify shapefile to include styles
    modify_shapefile_style(input_dir)
    
    # Register PDF driver
    gdal.GetDriverByName('PDF').Register()
    
    # Set up options for PDF creation
    translate_options = gdal.TranslateOptions(
        format="PDF",
        creationOptions=[
            f"OGR_DATASOURCE={input_dir}",
            "OGR_DISPLAY_FIELD=class",
            "OGR_DISPLAY_LAYER=segments",
            "LAYER_NAME=Ortho Image",
            "EXTRA_LAYER_NAME=Vector Overlay",
            "PDF_LAYER_ORDER=ON",
            "OGR_PDF_WRITE_INFO=ON",
            "DPI=300",
            "GEO_ENCODING=ISO32000",
            "MARGIN=0",
            "EXTRA_STREAM=OPACITY:100"
        ]
    )
    
    # Create PDF with TIFF and vectors
    gdal.Translate(output_pdf, input_tiff, options=translate_options)

def main():
    input_tiff = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\1627_utm\P2_35A_imagesRGB_orthomosaic.tif"
    input_directory = r"C:\Users\User\Downloads\1627_utm_checked\1627_utm_checked\Results\vectors"
    output_pdf = "styled_georef.pdf"
    
    create_georef_pdf(input_tiff, input_directory, output_pdf)

if __name__ == "__main__":
    main()

In [8]:
from osgeo import gdal
print(gdal.__version__)

3.4.3
